In [1]:
import sctop as top
import pandas as pd
from taigapy import create_taiga_client_v3
import numpy as np
import io
import os
import seaborn as sns

from numpy.random import default_rng

rng = default_rng(2024)

tc = create_taiga_client_v3()
import matplotlib.pyplot as plt
 

In [13]:
class OrderParamClassifier:
    def __init__(self, expr, markers, num_samples):
        """

        :param expr: Expression data.
            Format: Pandas DataFrame.
            Rows: samples (index)
            Columns: genes (ensg).
            Entries: log2(tpm+1)
        :param markers: Cell types
            Format: Pandas Series
            Rows: samples (index)
            Columns: metadata
            Entries: plain text. at least one cell/disease type column
        :param num_samples: Int. Number of samples to take from each marker for training

        """
        self.expr = expr
        self.markers = markers
        self.num_samples = num_samples
        self.basis = pd.DataFrame()

        self.harmonize_samples()

    def load_expr(self):
        pass

    def load_meta(self):
        pass

    def harmonize_samples(self):
        shared_idxs = self.expr.index.intersection(self.markers.index)
        self.expr = self.expr.loc[shared_idxs]
        self.markers = self.markers.loc[shared_idxs]

        marker_counts = self.markers.value_counts()
        valid_markers = marker_counts[marker_counts > self.num_samples].index
        self.markers = self.markers[self.markers.isin(valid_markers)]

    def train(self):
        expr_list = []
        train_ids = []

        labels = self.markers.unique().tolist()

        for curr_type in labels:
            curr_type_ids = self.markers[self.markers == curr_type].index
            train_sample = rng.choice(curr_type_ids, self.num_samples, replace=False)
            train_ids += [train_sample]

            curr_expr = self.expr.loc[train_sample]
            curr_processed = top.process(2 ** curr_expr.T - 1, average=True)
            expr_list.append(curr_processed)

        train_ids = np.concatenate(train_ids)
        self.basis = pd.concat(expr_list, axis=1)
        self.basis.columns = labels
        self.basis.index = self.basis.index.rename('gene')

        accuracies, test_labels, projections, eta = self.validate_training(train_ids)
        return train_ids, accuracies, test_labels, projections, eta

    def score(self, test_data):
        processed = top.process(2 ** test_data.T - 1)
        projections, _, eta = top.score(self.basis, processed, full_output=True)
        return projections, eta

    def validate_training(self, train_ids):
        test_ids = self.markers.index.difference(self.markers.loc[train_ids].index)
        projections, eta = self.score(self.expr.loc[test_ids])

        return self.check_acc(projections) + (projections, eta)

    def check_acc(self, projections, skip_fun=None, specification_value=0.1):
        if not skip_fun:
            def skip_fun(sample, ann):
                return False

        accuracies = {'top1': 0,
                      'top3': 0,
                      'unspecified': 0
                      }

        num_tested = 0

        labels = []

        for sample_id, sample_projections in projections.iteritems():

            types_sorted_by_projections = sample_projections.sort_values(ascending=False).index
            true_type = self.markers.loc[sample_id]

            if skip_fun(sample_id, self.markers):
                continue

            num_tested += 1

            top_type = types_sorted_by_projections[0]

            if sample_projections.max() < specification_value:
                accuracies['unspecified'] += 1

            if top_type == true_type:
                accuracies['top1'] += 1
            if true_type in types_sorted_by_projections[:3]:
                accuracies['top3'] += 1

            true_idx_tup = (true_type, true_type, true_type)
            label_df = pd.DataFrame(data=true_idx_tup, columns=['true_type'])
            label_df['labeled_type'] = types_sorted_by_projections[:3]
            label_df.index.name = 'place'
            labels.append(label_df.reset_index())

        print('Accuracy statistics for {} test samples and {} training samples per cancer type'.format(num_tested,
                                                                                                       self.num_samples))

        for key, value in accuracies.items():
            print('{}: {:.2f}%'.format(key, 100 * value / num_tested))

        return accuracies, pd.concat(labels, ignore_index=True)


def get_gene_map(tc, get_transcripts=False):
    hgnc = tc.get(name='hgnc-87ab', version=7, file='hgnc_complete_set')
    hgnc = hgnc[['hgnc_id', 'symbol', 'locus_group', 'locus_type', 'entrez_id', 'ensembl_gene_id']]
    hgnc['depmap'] = hgnc.symbol + ' (' + hgnc.entrez_id.fillna(-1).astype(int).astype(str) + ')'
    hgnc.loc[(hgnc.symbol.isna()) | (hgnc.entrez_id.isna()), 'depmap'] = np.nan
    if get_transcripts:
        ensembl = generateGeneNames(attributes=['ensembl_transcript_id'], cache_folder=os.getcwd() + '/ensemblecache/')
        print(ensembl)
        hgnc = pd.merge(hgnc,
                        ensembl.set_index('ensembl_gene_id').ensembl_transcript_id,
                        left_on='ensembl_gene_id', right_index=True)
    return hgnc


def get_tcga_classifier(num_samples, annots=None, marker=None):
    tcga_expr = tc.get('celligner-input-9827.1/tumor_expression').fillna(0)
    if annots is None:
        annots = tc.get('celligner-input-9827.7/tumor_annotations').set_index('sampleID')
    if marker is None:
        tcga_ann = annots.oncotree_lineage
    else:
        tcga_ann = annots[marker]

    cfier = OrderParamClassifier(tcga_expr, tcga_ann, num_samples)
    return cfier


oncotree = tc.get(name="oncotree-42c7", version=1, file="oncotree").set_index("code")
oncotree=oncotree.reset_index()

# Step 1: Find names with more than one unique OncoTree code
multiple_codes = oncotree.groupby('name')['code'].nunique()
names_with_multiple_codes = multiple_codes[multiple_codes > 1].index

# Step 2: Update the 'name' column in the oncotree DataFrame
def update_name(row):
    if row['name'] in names_with_multiple_codes:
        return f"{row['name']} ({row['tissue']})"
    return row['name']

oncotree['name'] = oncotree.apply(update_name, axis=1)
oncotree=oncotree.set_index('code')

    
def get_tcga_classifier_bysubtype(num_samples):
    tcga_expr = tc.get('celligner-input-9827.1/tumor_expression').fillna(0)
    tcga_meta=pd.read_csv('/Users/nabdirah/Project1/3TCGA.csv') # with the new mapped subtypes 
    
    tcga_ann = tcga_meta.set_index('sampleID').new_subtype 

    cfier = OrderParamClassifier(tcga_expr, tcga_ann, num_samples)
    return cfier
    


def get_gtext_classifier():
    pass



def generateGeneNames(
    ensemble_server="http://nov2020.archive.ensembl.org/biomart",
    useCache=False,
    cache_folder='/home/ubuntu/Documents/data/ensemblcache/',
    attributes=[],
    default_attr=[
        "ensembl_gene_id",
        "clone_based_ensembl_gene",
        "hgnc_symbol",
        "gene_biotype",
        "entrezgene_id",
    ],
):
    """generate a genelist dataframe from ensembl's biomart

    Args:
        ensemble_server ([type], optional): [description]. Defaults to ENSEMBL_SERVER_V.
        useCache (bool, optional): [description]. Defaults to False.
        cache_folder ([type], optional): [description]. Defaults to CACHE_PATH.

    Raises:
        ValueError: [description]

    Returns:
        [type]: [description]
    """
    assert cache_folder[-1] == "/"

    cache_folder = os.path.expanduser(cache_folder)
    createFoldersFor(cache_folder)
    cachefile = os.path.join(cache_folder, ".biomart.csv")
    if useCache & os.path.isfile(cachefile):
        print("fetching gene names from biomart cache")
        res = pd.read_csv(cachefile)
    else:
        print("downloading gene names from biomart")
        res = _fetchFromServer(ensemble_server, default_attr + attributes)
        res.to_csv(cachefile, index=False)

    res.columns = default_attr + attributes
    if type(res) is not type(pd.DataFrame()):
        raise ValueError("should be a dataframe")

    if "clone_based_ensembl_gene" in default_attr and "hgnc_symbol" in default_attr:
        res = res[~(res["clone_based_ensembl_gene"].isna() & res["hgnc_symbol"].isna())]
        res.loc[res[res.hgnc_symbol.isna()].index, "hgnc_symbol"] = res[
            res.hgnc_symbol.isna()
        ]["clone_based_ensembl_gene"]

    return res

def createFoldersFor(filepath):
    """
    will recursively create folders if needed until having all the folders required to save the file in this filepath
    """
    prevval = ""
    for val in os.path.expanduser(filepath).split("/")[:-1]:
        prevval += val + "/"
        if not os.path.exists(prevval):
            os.mkdir(prevval)

def _fetchFromServer(ensemble_server, attributes):
    # deferring the import until last possible moment because it's unclear
    # to me whether we are actually fetching data from biomart. (I don't think we should)
    from biomart import BiomartServer

    server = BiomartServer(ensemble_server)
    ensmbl = server.datasets["hsapiens_gene_ensembl"]
    res = pd.read_csv(
        io.StringIO(
            ensmbl.search({"attributes": attributes}, header=1).content.decode()
        ),
        sep="\t",
    )
    return res


In [21]:
tcga_meta=pd.read_csv('/Users/nabdirah/Project1/3TCGA.csv')

In [17]:
# Step 2: Update the 'name' column in the oncotree DataFrame
oncotree=oncotree.set_index('code')
def update_name(row):
    if row['name'] in names_with_multiple_codes:
        return f"{row['name']} ({row['tissue']})"
    return row['name']

oncotree['name'] = oncotree.apply(update_name, axis=1)
oncotree=oncotree.set_index('code')
glioma_row = oncotree.loc['GNOS']
glioma_row['name'] = 'glioma'
glioma_row.name = 'GNOS_1'
oncotree=pd.concat([oncotree,glioma_row.to_frame().T])
acute_Row=oncotree.loc['TLL']	
acute_Row['name'] = 'acute lymphoblastic leukemia'
acute_Row.name = 'TLL_1'
ocotree=pd.concat([oncotree,acute_Row.to_frame().T])

In [22]:
sample_sizes = 25
classifier = get_tcga_classifier_bysubtype(sample_sizes) 

In [23]:
classifier.train()

/var/folders/p8/ns8k_3ds56130jc48rg_027h0000gp/T/ipykernel_56106/2875311688.py:88: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for sample_id, sample_projections in projections.iteritems():


Accuracy statistics for 10478 test samples and 25 training samples per cancer type
top1: 84.53%
top3: 95.32%
unspecified: 0.00%


(array(['TH02_0085_S01', 'THR24_1970_S01', 'THR11_0245_S01',
        'TCGA-19-2619-01', 'THR24_1976_S01', 'TCGA-TQ-A7RN-01',
        'TCGA-DU-6392-01', 'TCGA-S9-A7J2-01', 'TCGA-27-1837-01',
        'THR24_2028_S01', 'TCGA-DB-5277-01', 'TCGA-TQ-A7RO-01',
        'TCGA-DB-A64S-01', 'THR24_2068_S01', 'THR11_0267_S01',
        'THR24_1927_S01', 'TCGA-02-0055-01', 'THR13_0974_S01',
        'TH02_0095_S01', 'TCGA-19-5960-01', 'TCGA-HT-7881-01',
        'THR24_1997_S01', 'THR14_0303_S01', 'TCGA-19-2629-01',
        'TCGA-DU-7300-01', 'TARGET-40-PATPBS-01A-01R', 'THR25_0635_S01',
        'THR15_0348_S01', 'TH27_0703_S02', 'TARGET-40-PARFTG-01A-01R',
        'TH38_1416_S02', 'THR15_0345_S01', 'TH03_0296_S01',
        'TARGET-40-0A4I65-01A-01R', 'TH27_1273_S01', 'TH06_0630_S01',
        'THR25_0637_S01', 'TH27_0698_S01', 'THR15_0356_S01',
        'TARGET-40-PANSEN-01A-01R', 'TH06_0634_S01', 'THR24_2089_S01',
        'THR15_0357_S01', 'THR31_0893_S01', 'TARGET-40-PARBGW-01A-01R',
        'TH03_00

In [ ]:
"""performs an iterative process to train the classifier, calculate the accuracy for different subtypes based on top predictions, 
and store the results in a dictionary."""
acc_dict = dict()
sample_sizes = [25]
for ssize in sample_sizes:
    
    
    accuracy_list = []
    for i in range(5):
        classifier = get_tcga_classifier_bysubtype(ssize) 
        train_ids, accuracies, test_labels, projections, eta = classifier.train()
        score_data=projections, eta
        Top_score=score_data[0].apply(lambda x: x.sort_values(ascending=False).iloc[:3].index,axis = 0)
        data=Top_score.T
        df=tcga_meta[['sampleID','new_subtype']] 
        table=pd.merge(data,df, right_on='sampleID', left_index=True)
        #table = pd.merge(df, data, left_on='sampleID', right_index=True)
        grouped=table.groupby(['new_subtype'])
        
        accuracy=grouped.apply(lambda _df: _df.apply(lambda row:row.new_subtype in row.loc[[0,1,2]].tolist(),axis=1).mean())
        #print("The Standard Deviation of the thth accuracy is",accuracy.std())
        accuracy=pd.DataFrame(accuracy)
        accuracy.apply(lambda x: x.sort_values(ascending=False),axis = 0)
        accuracy_list.append(accuracy)
    acc_dict[ssize] = accuracy_list

In [ ]:
classifier.markers.value_counts()

In [27]:
# Reset the index for each DataFrame in the list `acc_dict[25]`
dfs = [s.reset_index() for s in acc_dict[25]]

# Concatenate all DataFrames in the list `dfs` into a single DataFrame
combined_acc_df = pd.concat(dfs, ignore_index=True)

# Rename the columns of the combined DataFrame
combined_acc_df.columns = ['new_subtype', 'value']


In [ ]:
def top_common_subtypes(df):
    """gets the 3 most common subtypes predicted for each subtype"""
    
    subtypes =df.loc[:, [0,1,2]].values.flatten()  
    subtypes = pd.DataFrame(subtypes, columns=['top_subtypes'])
    top3 = subtypes['top_subtypes'].value_counts().head(3)
    return top3
    
topsubtype=[]
for subtype in table.new_subtype.unique():
    subtype_df =table[table['new_subtype'] == subtype]
    top_df = top_common_subtypes(subtype_df)
    topsubtype.append((subtype, top_df))

In [36]:
# Concatenate DataFrames for each subtype into a single DataFrame
pair_df = pd.concat([pd.concat({topsubtype[i][0]: pd.DataFrame(data=topsubtype[i][1])}, names=['annotated_subtype']) for i in range(len(topsubtype))])


In [31]:
def get_pair(correlationdata, filtereddata, subtypes):
    """
    Generates pairs of subtypes and their corresponding correlation matrices.

    Parameters:
    - correlationdata (pd.DataFrame): DataFrame with correlation values between samples.
    - filtereddata (pd.DataFrame): DataFrame with filtered sample data.
    - subtypes (list): List of subtype names.

    Returns:
    - list: A list of tuples where each tuple contains two subtypes and their correlation matrix.
    """
    subtype_pairs = []
    first_subtype = subtypes[0]  # Use the first subtype as the reference for comparisons
    
    for subtype2 in subtypes:
        # Get the sample IDs for the current subtype and the reference subtype
        samples1 = filtereddata[filtereddata['new_subtype'] == first_subtype]['sampleID']
        samples2 = filtereddata[filtereddata['new_subtype'] == subtype2]['sampleID']
        
        # Extract the correlation matrix for the pair of subtypes
        comparison = correlationdata.loc[samples1, samples2]
        
        # Append the subtype pair and their correlation matrix to the list
        subtype_pairs.append((first_subtype, subtype2, comparison))
    
    return subtype_pairs

def get_pair_stat(subtype_pairs, correlation_data, filtered_data):
    """
    Computes the median correlation for each pair of subtypes.

    Parameters:
    - subtype_pairs (list): List of tuples containing subtype pairs and their correlation matrices.
    - correlation_data (pd.DataFrame): DataFrame with correlation values between samples.
    - filtered_data (pd.DataFrame): DataFrame with filtered sample data.

    Returns:
    - pd.DataFrame: DataFrame with subtype pairs and their median correlation values.
    """
    results = []
    
    for subtype1, subtype2, comparison in subtype_pairs:
        if not comparison.empty:
            if subtype1 == subtype2:
                # Compute median correlation for the lower triangle of the matrix if subtypes are the same
                tril = np.tril_indices(len(comparison), -1)
                median_corr = np.median(comparison.to_numpy()[tril])
            else:
                # Compute median correlation for the entire matrix if subtypes are different
                median_corr = np.median(comparison.to_numpy().flatten())
        
            # Append the result as a tuple (subtype1, subtype2, median_correlation) to the results list
            results.append((subtype1, subtype2, median_corr))
        else:
            # If the comparison matrix is empty, append NaNs for median correlation
            results.append((subtype1, subtype2, float('nan'), float('nan')))

    # Convert the results list to a DataFrame with specified column names
    return pd.DataFrame(results, columns=['col1', 'col2', 'median_correlation'])
    
Dataf=pair_df.reset_index()
def get_substat(subtype, Data):
    """
    Gets statistics for a given subtype including top subtypes, correlation data, and median correlations.

    Parameters:
    - subtype (str): The subtype for which statistics are being computed.
    - Data (pd.DataFrame): DataFrame containing annotated_subtype and its top3 subtypes.

    Returns:
    - pd.DataFrame: DataFrame with statistics for the given subtype.
    """
    print(f"Getting top subtypes for {subtype}")
    # Filter the data for the given subtype and get unique levels
    topsubtypes = Data[Data.annotated_subtype == subtype].level_1.unique()
    
    # Filter tcga_meta DataFrame to include only the top subtypes
    filtered_data = tcga_meta[tcga_meta['new_subtype'].isin(topsubtypes)]
    
    print("Getting correlation data")
    # Compute correlation matrix from classifier's expression data for the filtered samples
    correlation_data = classifier.expr.loc[filtered_data['sampleID']].T.corr()
    
    # Get pairs of subtypes and their correlation matrices
    pair = get_pair(correlation_data, filtered_data, topsubtypes)
    print(f"Number of subtypes processed: {len(subtype)}")

    print('Getting statistics')
    # Compute median correlations for the subtype pairs
    stat = get_pair_stat(pair, correlation_data, filtered_data)
    
    return pd.DataFrame(stat)



In [ ]:
subtype_table=[]
for subtype in Data.annotated_subtype.unique():
    subtype_df=get_substat(subtype, Data)
    subtype_table.append(subtype_df)

In [ ]:
df_sub_corr = pd.concat(subtype_table)
df_sub_corr.rename(columns={'col1':'annotated_subtype','col2':'top3'}, inplace=True)
df_sub_corr.to_csv('/Users/nabdirah/Project1/newsub_correlation.csv')


In [ ]:
# for each annotated_subtype find the score the model gave to its top3 subtypes 
def get_subtypescore(subtype):
    """Parameters:
    - subtype (str): The subtype for which to calculate the scores.

    Returns:
    - pd.Series: A Series with the average score for each sample in the subset.
    """
    print(subtype)
    sample_set_bl = classifier.markers[classifier.markers == subtype].index
    print(sample_set_bl)
    subset = projections.loc[pair_df.loc[(subtype,),:].index,(sample_set_bl).intersection(projections.columns.tolist())]
    print(subset)
    Aug_score =subset.mean(axis=1)
    return Aug_score
subscore_df=[]
for subtype in classifier.markers.unique():
    subscore=get_subtypescore(subtype)
    subscore_df.append((subtype, subscore))



In [ ]:
#Concatenate all the subtypes and their scores into a single DataFrame
score_df = pd.concat([pd.concat({subscore_df[i][0]: pd.DataFrame(data=subscore_df[i][1])}, names=['annotated_subtype']) for i in range(len(subscore_df))])

In [ ]:
score_df=score_df.reset_index()
score_df.rename(columns={'level_1': 'top3', 0:'score'}, inplace=True)
score_df

In [ ]:
#merge the Dataframes with scores and median correlation 
merged_dfs=pd.merge(score_df,df_sub_corr,left_on=['annotated_subtype','top3'],right_on=['annotated_subtype','top3'])

In [ ]:
#save
merged_dfs.to_csv('/Users/nabdirah/Project1/correlationscore.csv')
merged_dfs.drop(columns='level_0')
merged_dfs.set_index('annotated_subtype')
merged_dfs = merged_dfs.drop(columns=['level_0', 'index'])
merged_dfs.rename(columns={0:'score'}, inplace=True)
merged_dfs=merged_dfs.drop_duplicates()
merged_dfs.to_csv('/Users/nabdirah/Project1/Fixed_data.csv')

In [ ]:
# list of value counts for the different subtypes we have at this point
subtpe_count=[merged_dfs['annotated_subtype'].value_counts()]
subtpe_count